In [ ]:
%load_ext autoreload
%autoreload 2

import auxiliary.util as util
import pickle as pkl
import numpy as np
import random
from typing import Union, List, Tuple, Optional

from torch_geometric.data import Dataset
#import imitation_data_loading
#from tutor_data_loading import preprocess_observation, action_identificator, Episode, EpisodeStep, EpisodeSet, StepSet
import torch
import os
import json
import random
import wandb
import training.training as trn
from training.models import GCN, FCNN
from training.dataloader import TutorDataLoader
import collections
import matplotlib.pyplot as plt

# Data Loading

In [ ]:
util.set_wd_to_package_root()
config = util.load_config()
processed_data_path = config['paths']['processed_tutor_imitation']
matrix_cache_path = config['paths']['con_matrix_cache']
feature_statistics_path = config['paths']['feature_statistics']
action_counter_path = config['paths']['action_counter']

# Training Loop

In [ ]:
#todo seperate static (line capacities, 'standard' voltage) and dynamic attributes
#compare weights or/ex layers (sign?)
#feature: number of connected nodes?\

In [ ]:
training_config = util.load_config()['training']

In [ ]:
training_config

In [ ]:
%%wandb
import matplotlib.pyplot as plt



estimated_train_size = int(0.8*25993)
label_smth_alpha = training_config['label_smoothing_alpha']


with tqdm(total=n_epoch*estimated_train_size) as pbar:
    model.zero_grad()
    step=0
    for e in range(n_epoch):
        for datapoint in train_dl:
            
            #Get information from datapoint

            

            






                

                


In [ ]:
norm_W_sb_neigh


In [ ]:
model.lin_gen_1.weight

In [ ]:
wrong_indices

In [ ]:
print(abs(model.GNN_layers[6].convs['object__other_busbar__object'].lin_l.weight).sum())
print(abs(model.GNN_layers[6].convs['object__same_busbar__object'].lin_r.weight).sum())

In [ ]:
model.GNN_layers[6].convs.values()

In [ ]:
diffs = []
for l in model.GNN_layers:
    print(l)
    if training_config['network_type'] == 'heterogenous':
        norm_W_self = abs(l.convs['object__same_busbar__object'].lin_r.weight).sum()
        norm_W_neigh = abs(l.convs['object__line__object'].lin_l.weight).sum() + \
                        abs(l.convs['object__same_busbar__object'].lin_l.weight).sum() + \
                        abs(l.convs['object__other_busbar__object'].lin_l.weight).sum()
        diffs.append(norm_W_self-norm_W_neigh)
    else:
        assert False, 'Not developped yet'

In [ ]:
diffs

In [ ]:
model.GNN_layers[6].convs['object__same_busbar__object'].forward(torch.rand((1,32),device=device),
                                                                  torch.tensor([],dtype=torch.long,device=device).reshape(2,0))

In [ ]:
model.GNN_layers[6].convs['object__same_busbar__object'].lin_l.bias

In [ ]:
training_config['network_type']

# Feature importance analysis with SHAP

In [ ]:
import shap

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available()
                                   else 'cpu')

In [ ]:
class GCNAdapter(GCN):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def forward()
        
    

In [ ]:
train_config = config['training']
if config['training']['hyperparams']['model_type'] == 'GCN':
    model = GCNAdapter(train_config['hyperparams']['LReLu_neg_slope'],
                         train_config['hyperparams']['weight_init_std'],
                         train_config['GCN']['constants']['N_f_gen'],
                         train_config['GCN']['constants']['N_f_load'],
                         train_config['GCN']['constants']['N_f_endpoint'],
                         train_config['GCN']['hyperparams']['N_GCN_layers'],
                         train_config['hyperparams']['N_node_hidden'],
                         train_config['GCN']['hyperparams']['aggr'],
                         train_config['GCN']['hyperparams']['network_type'])
else: 
    model = FCNN(train_config['hyperparams']['LReLu_neg_slope'],
                                  train_config['hyperparams']['weight_init_std'],
                                  train_config['FCNN']['constants']['size_in'],
                                  train_config['FCNN']['constants']['size_out'],
                                  train_config['FCNN']['hyperparams']['N_layers'],
                                  train_config['hyperparams']['N_node_hidden'])
model.load_state_dict(torch.load('models/FCNN_test',map_location=torch.device('cpu')))
model.eval()

In [ ]:
if config['training']['hyperparams']['model_type'] == 'GCN':
    pass
else:
    val_dl = TutorDataLoader(processed_data_path + '/val', 
                 matrix_cache_path, 
                 feature_statistics_path, 
                 action_counter_path,
                 device, 
                 FCNN,
                 None,
                 False,
                 0)
    
    dps = torch.Tensor(0,config['training']['FCNN']['constants']['size_in'])

    val_dl_iter = val_dl.__iter__()
    for i in range(100):
        dp = next(val_dl_iter)
        dps = torch.cat((dps,dp['features'].reshape(1,-1)),axis=0)

e = shap.DeepExplainer(model, dps)
shap_values = e.shap_values(dps)
mean_shap_over_dps = np.array(shap_values).mean(axis=(1))
    


In [ ]:
mean_shap_over_dps
N_gen = 5
N_f_gen= 3 #Number of generator features
N_load = 11
N_f_load= 3 #Number of load features
N_line = 20
N_f_endpoint= 6 #Number of endpoint (origin/extremity) features
       
c = 0
mean_shap_x_gen = mean_shap_over_dps[:,0:N_gen*N_f_gen]
c+=N_gen*N_f_gen
mean_shap_x_load = mean_shap_over_dps[:,c:c+N_load*N_f_load]
c+=N_load*N_f_load
mean_shap_x_or = mean_shap_over_dps[:,c:c+N_line*N_f_endpoint]
c+=N_line*N_f_endpoint
mean_shap_x_ex = mean_shap_over_dps[:,c:c+N_line*N_f_endpoint]
c+=N_line*N_f_endpoint
mean_shap_x_topo_vect = mean_shap_over_dps[:,c:]

In [ ]:
mean_shap_x_gen = mean_shap_x_gen.transpose().reshape(N_gen,N_f_gen,-1)
mean_shap_x_load = mean_shap_x_load.transpose().reshape(N_load,N_f_load,-1)
mean_shap_x_or = mean_shap_x_or.transpose().reshape(N_line,N_f_endpoint,-1)
mean_shap_x_ex = mean_shap_x_ex.transpose().reshape(N_line,N_f_endpoint,-1)
mean_shap_x_topo_vect = mean_shap_x_topo_vect.transpose()

## Plots

In [ ]:
plt.bar(range(len(e.expected_value)),e.expected_value)

In [ ]:
fig, axs = plt.subplots(3, 2)
fig.set_size_inches(18.5, 10.5)
axs[0,0].bar(['p','q','v'],mean_shap_x_gen.mean(axis=(0,2)))
axs[0,1].bar(['p','q','v'],mean_shap_x_load.mean(axis=(0,2)))
axs[1,0].bar(['p', 'q', 'v', 'a', 'line_rho','line_capacity'],mean_shap_x_or.mean(axis=(0,2)))
axs[1,1].bar(['p', 'q', 'v', 'a', 'line_rho','line_capacity'],mean_shap_x_ex.mean(axis=(0,2)))
axs[2,0].bar(range(len(mean_shap_x_topo_vect)),mean_shap_x_topo_vect.mean(axis=1))

In [ ]:
import matplotlib.gridspec as gridspec
import seaborn as sns

cumulative_sub_info = np.cumsum([3, 6, 4, 6, 5, 6, 3, 2, 5, 3, 3, 3, 4, 3])
grid = np.concatenate([mean_shap_x_gen.mean(axis=1),
                       mean_shap_x_load.mean(axis=1),
                       mean_shap_x_or.mean(axis=1),
                       mean_shap_x_ex.mean(axis=1)])
ax = sns.heatmap(grid, cmap="YlGnBu")
fig.set_size_inches(18.5, 10.5)
ax.set(xlabel='Output object', ylabel='Input object', title='Mean shapley values')
ax.vlines(cumulative_sub_info, *ax.get_xlim())
ax.hlines(cumulative_sub_info, *ax.get_xlim())

In [ ]:
len(cumulative_sub_info)

In [ ]:
grid = mean_shap_x_topo_vect
ax = sns.heatmap(grid, cmap="YlGnBu")
fig.set_size_inches(18.5, 10.5)
ax.set(xlabel='Output object', ylabel='Input topo vect index', title='Mean shapley values')
ax.vlines(cumulative_sub_info, *ax.get_xlim())
ax.hlines(cumulative_sub_info, *ax.get_xlim())

In [ ]:
grid

In [ ]:
grid.shape

In [ ]:
mean_shap_x_gen

In [ ]:
torch.mean(dps,axis=0)

# Testing with Jacobian

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available()
                                   else 'cpu')

In [ ]:
train_config = config['training']
if config['training']['hyperparams']['model_type'] == 'GCN':
    model = GCN(train_config['hyperparams']['LReLu_neg_slope'],
                 train_config['hyperparams']['weight_init_std'],
                 train_config['GCN']['constants']['N_f_gen'],
                 train_config['GCN']['constants']['N_f_load'],
                 train_config['GCN']['constants']['N_f_endpoint'],
                 train_config['GCN']['hyperparams']['N_GNN_layers'],
                 train_config['hyperparams']['N_node_hidden'],
                 train_config['GCN']['hyperparams']['aggr'],
                 train_config['GCN']['hyperparams']['network_type'])
else: 
    model = FCNN(train_config['hyperparams']['LReLu_neg_slope'],
                                  train_config['hyperparams']['weight_init_std'],
                                  train_config['FCNN']['constants']['size_in'],
                                  train_config['FCNN']['constants']['size_out'],
                                  train_config['FCNN']['hyperparams']['N_layers'],
                                  train_config['hyperparams']['N_node_hidden'])
model.load_state_dict(torch.load('models/FCNN_test',map_location=torch.device('cpu')))
model.eval()

In [ ]:
val_dl = TutorDataLoader(processed_data_path + '/train', 
                 matrix_cache_path, 
                 feature_statistics_path, 
                 action_counter_path,
                 device, 
                 FCNN,
                 None,
                 False,
                 0)
        

In [ ]:
jacobs_gen_mean = torch.zeros(3)
jacobs_load_mean = torch.zeros(3)
jacobs_or_mean = torch.zeros(6)
jacobs_ex_mean = torch.zeros(6)
jacobs_object_sum = torch.zeros((56,56))

Y_sum = torch.zeros(56)
for i in range(100):
    dp = next(val_dl.__iter__())
    print(dp)
    
    if type(model) == GCN:
        # Extract features
        X_gen = dp['gen_features']
        X_load = dp['load_features']
        X_or = dp['or_features']
        X_ex = dp['ex_features']

        # Extract the position topology vector, which relates the
        # objects ordered by type to their position in the topology vector
        object_ptv = dp['object_ptv']

        # Extract the edges
        E = dp['edges']

        # Pass through the model
        P = model(X_gen, X_load, X_or, X_ex, E, object_ptv).reshape((-1))
    elif type(model) == FCNN:
        # Pass through the model
        P = model(dp['features']).reshape((-1))

    #Extract the label, apply label smoothing
    Y = dp['change_topo_vect']
    label_smth_alpha = train_config['hyperparams'] \
                                        ['label_smoothing_alpha']
    Y_smth =(1-label_smth_alpha)*dp['change_topo_vect'] + \
              label_smth_alpha*0.5*torch.ones_like(Y,device=device)
    Y_sum += Y
    
    #Compute the weights for the loss
    non_sub_label_weight = train_config['hyperparams'] \
                                        ['non_sub_label_weight']
    Y_sub_mask, Y_sub_idx = trn.get_Y_subchanged(Y,dp['sub_info'])
    weights = trn.label_weights(1-Y_sub_mask,non_sub_label_weight)
    
    
    
    #Compute the loss, update gradients
    l = trn.BCELoss_labels_weighted(P,Y_smth,weights)
    l.backward()
    
    #Obtain the jacobian
    
    jacob = torch.autograd.functional.jacobian(func:= lambda xg, xl, xo, xe: \
                                   model(xg,xl,xo,xe,E,object_ptv),
                                   inputs=(X_gen, X_load, X_or, X_ex))
    
    #Aggregate the jacobian to determine the importance of each feature
    jacobs_gen_mean += torch.mean(torch.abs(out[0]),axis=(0,1,2))
    jacobs_load_mean += torch.mean(torch.abs(out[1]),axis=(0,1,2))
    jacobs_or_mean += torch.mean(torch.abs(out[2]),axis=(0,1,2))
    jacobs_ex_mean += torch.mean(torch.abs(out[3]),axis=(0,1,2))
    #jacobs_or_sum = torch.zeros(6)
    #jacobs_ex_sum = torch.zeros(6)
    
    #Aggregate the jacobian to determine the importance of each object's features to each objects output
    jacob_objects = torch.cat([torch.sum(torch.abs(out[0]),axis=(1,3)),
        torch.sum(torch.abs(out[1]),axis=(1,3)),
        torch.sum(torch.abs(out[2]),axis=(1,3)),
        torch.sum(torch.abs(out[3]),axis=(1,3))],axis=1)[:,object_ptv]
    jacobs_object_sum += jacob_objects

In [ ]:
fig, axs = plt.subplots(2, 2)
fig.set_size_inches(18.5, 10.5)
axs[0,0].bar(['p','q','v'],jacobs_gen_mean)
axs[0,1].bar(['p','q','v'],jacobs_load_mean)
axs[1,0].bar(['p', 'q', 'v', 'a', 'line_rho','line_capacity'],jacobs_or_mean)
axs[1,1].bar(['p', 'q', 'v', 'a', 'line_rho','line_capacity'],jacobs_ex_mean)
#axs[0,0].bar(['p','q','v'],jacobs_gen_mean)

In [ ]:
import matplotlib.gridspec as gridspec
# gs = gridspec.GridSpec(1,2, height_ratios=[1,1], width_ratios=[1])
# print(gs)
# f = plt.figure()
# ax1 = plt.subplot(gs[0])
# ax2 = plt.subplot(gs[1])

fig, axs = plt.subplots(1,2, gridspec_kw={'width_ratios': [0.65, 1]})
fig.set_size_inches(18.5, 10.5)
#f.subplots_adjust(hspace=0)
axs[0].imshow(torch.log(Y_sum.reshape(-1,1)+0.1))
axs[1].imshow(jacobs_sum)

In [ ]:
torch.argmax(jacobs_sum.sum(axis=0))

# Old

#### Testing whether the ordering checks out

In [ ]:
ds = TutorDataset(processed_data_path, matrix_cache_path, feature_statistics_path)
dp = next(iter(ds))

In [ ]:
gen_ptv = dp['gen_pos_topo_vect']
load_ptv = dp['load_pos_topo_vect']
or_ptv = dp['line_or_pos_topo_vect']
ex_ptv = dp['line_ex_pos_topo_vect']
object_indices = np.argsort(np.concatenate([gen_ptv,load_ptv,or_ptv,ex_ptv]))

In [ ]:
gen_ptv

In [ ]:
object_indices

In [ ]:
gen_dummy = len(gen_ptv)*[0]
gen_dummy[-1] = 1000
load_dummy = len(load_ptv)*[1]
or_dummy = len(or_ptv)*[2]
ex_dummy = len(ex_ptv)*[3]
dummies = np.array(gen_dummy + load_dummy + or_dummy + ex_dummy)
dummies[object_indices]

In [ ]:
dummies_tensor = torch.cat([torch.tensor(d) for d in [gen_dummy,load_dummy,or_dummy,ex_dummy]])
dummies_tensor[object_indices]